In [2]:
import os
import pandas as pd
import numpy as np
import logging
import datetime

import re
import sys
import concurrent.futures
import multiprocessing
num_processes = multiprocessing.cpu_count() - 2

In [58]:
num_processes

22

In [4]:
compare = "/home/laura/ANALYSIS/VARIANT_CALLING/Cluster459_autosnippy/Compare/2021-04-13_Cluster459_autosnippy/Cluster459_autosnippy.revised_intermediate.tsv"
variant_folder = "/home/laura/ANALYSIS/VARIANT_CALLING/Cluster459_autosnippy/Variants"

In [5]:
def check_create_dir(path):
    # exists = os.path.isfile(path)
    # exists = os.path.isdir(path)
    if os.path.exists(path):
        pass
    else:
        os.mkdir(path)

In [6]:
# LOGGING
# Create log file with date and time
right_now = str(datetime.datetime.now())
right_now_full = "_".join(right_now.split(" "))
log_filename = "/home/laura/ANALYSIS/MISC/Test_multithreading_vcf/test" + "_" + right_now_full + ".log"
log_folder = os.path.join(".", 'Logs')
check_create_dir(log_folder)
log_full_path = os.path.join(log_folder, log_filename)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s:%(message)s')

file_handler = logging.FileHandler(log_full_path)
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)

stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
# stream_handler.setFormatter(formatter)

logger.addHandler(stream_handler)
logger.addHandler(file_handler)

In [15]:
df = pd.read_csv(compare, sep="\t")
df[['CHROM','REF','POS', 'ALT']] = df.Position.str.split("|",expand=True,)
selected_columns = ['Position','N', 'Samples', 'CHROM','REF','POS', 'ALT']
df = df[ selected_columns + [ col for col in df.columns if col not in selected_columns]]


In [16]:
df.head()

,Position,N,Samples,CHROM,REF,POS,ALT,ALM12296078-20,ALM18609-08,ALM12247233-19,ALM64446-14,ALM29229-10,ALM12714-08,ALM12319442-20,ALM40310-07
0,MTB_anc|C|2532|T,8,"ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...",MTB_anc,C,2532,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,MTB_anc|C|9143|T,8,"ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...",MTB_anc,C,9143,T,0.99,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,MTB_anc|C|11370|T,8,"ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...",MTB_anc,C,11370,T,0.99,0.98,0.95,1.0,0.98,0.91,0.99,0.97
3,MTB_anc|G|12020|A,8,"ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...",MTB_anc,G,12020,A,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.99
4,MTB_anc|G|13460|A,8,"ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...",MTB_anc,G,13460,A,1.0,0.97,0.99,0.96,0.99,1.0,1.0,1.0


In [17]:
dft = df.transpose()

In [19]:
dft.head(10)

,0,1,2,3,4,5,6,7,8,9,...,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332
Position,MTB_anc|C|2532|T,MTB_anc|C|9143|T,MTB_anc|C|11370|T,MTB_anc|G|12020|A,MTB_anc|G|13460|A,MTB_anc|T|14051|C,MTB_anc|G|14401|A,MTB_anc|G|15117|C,MTB_anc|C|21819|A,MTB_anc|A|23174|C,...,MTB_anc|G|3943726|C,MTB_anc|ACCGGCGGCACCGGCGGCGGGGACGGGGGCGCCGGCGG...,MTB_anc|C|3945519|T,MTB_anc|C|3949453|T,MTB_anc|G|3949455|A,MTB_anc|G|4046255|A,MTB_anc|G|4262649|A,MTB_anc|C|4274930|T,MTB_anc|A|4305740|G,MTB_anc|T|4385568|C
N,8,8,8,8,8,7,8,8,8,8,...,1,2,1,1,1,1,1,1,1,1
Samples,"ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...",...,ALM40310-07,"ALM12296078-20,ALM40310-07",ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07
CHROM,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,...,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc
REF,C,C,C,G,G,T,G,G,C,A,...,G,ACCGGCGGCACCGGCGGCGGGGACGGGGGCGCCGGCGGCACCGGCGGCC,C,C,G,G,G,C,A,T
POS,2532,9143,11370,12020,13460,14051,14401,15117,21819,23174,...,3943726,3944442,3945519,3949453,3949455,4046255,4262649,4274930,4305740,4385568
ALT,T,T,T,A,A,C,A,C,A,C,...,C,A,T,T,A,A,A,T,G,C
ALM12296078-20,1.0,0.99,0.99,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,?,0,0,0,0,0,0,0,0
ALM18609-08,1.0,1.0,0.98,1.0,0.97,1.0,0.99,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
ALM12247233-19,1.0,1.0,0.95,1.0,0.99,1.0,1.0,1.0,?,1.0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
dft.loc[selected_columns,:]

,0,1,2,3,4,5,6,7,8,9,...,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332
Position,MTB_anc|C|2532|T,MTB_anc|C|9143|T,MTB_anc|C|11370|T,MTB_anc|G|12020|A,MTB_anc|G|13460|A,MTB_anc|T|14051|C,MTB_anc|G|14401|A,MTB_anc|G|15117|C,MTB_anc|C|21819|A,MTB_anc|A|23174|C,...,MTB_anc|G|3943726|C,MTB_anc|ACCGGCGGCACCGGCGGCGGGGACGGGGGCGCCGGCGG...,MTB_anc|C|3945519|T,MTB_anc|C|3949453|T,MTB_anc|G|3949455|A,MTB_anc|G|4046255|A,MTB_anc|G|4262649|A,MTB_anc|C|4274930|T,MTB_anc|A|4305740|G,MTB_anc|T|4385568|C
N,8,8,8,8,8,7,8,8,8,8,...,1,2,1,1,1,1,1,1,1,1
Samples,"ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...","ALM12296078-20,ALM18609-08,ALM12247233-19,ALM6...",...,ALM40310-07,"ALM12296078-20,ALM40310-07",ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07,ALM40310-07
CHROM,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,...,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc,MTB_anc
REF,C,C,C,G,G,T,G,G,C,A,...,G,ACCGGCGGCACCGGCGGCGGGGACGGGGGCGCCGGCGGCACCGGCGGCC,C,C,G,G,G,C,A,T
POS,2532,9143,11370,12020,13460,14051,14401,15117,21819,23174,...,3943726,3944442,3945519,3949453,3949455,4046255,4262649,4274930,4305740,4385568
ALT,T,T,T,A,A,C,A,C,A,C,...,C,A,T,T,A,A,A,T,G,C


In [48]:
dft.drop(selected_columns,axis=0)

,0,1,2,3,4,5,6,7,8,9,...,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332
ALM12296078-20,1.0,0.99,0.99,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0,?,0,0,0,0,0,0,0,0
ALM18609-08,1.0,1.0,0.98,1.0,0.97,1.0,0.99,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
ALM12247233-19,1.0,1.0,0.95,1.0,0.99,1.0,1.0,1.0,?,1.0,...,0,0,0,0,0,0,0,0,0,0
ALM64446-14,1.0,1.0,1.0,1.0,0.96,0.99,1.0,1.0,?,1.0,...,0,!,0,0,0,0,0,0,0,0
ALM29229-10,1.0,1.0,0.98,1.0,0.99,0.99,0.99,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
ALM12714-08,1.0,1.0,0.91,1.0,1.0,1.0,1.0,1.0,?,1.0,...,0,!,0,0,0,0,0,0,0,0
ALM12319442-20,1.0,1.0,0.99,1.0,1.0,1.0,1.0,1.0,?,0.99,...,0,0,0,0,0,0,0,0,0,0
ALM40310-07,1.0,1.0,0.97,0.99,1.0,0,1.0,1.0,?,0.99,...,1.0,?,1.0,?,?,1.0,0.99,1.0,1.0,1.0


In [25]:
POSs = dft.loc['POS',:].tolist()

In [37]:
len([0] * len(POSs)) == len(POSs)

True

In [28]:
POSs.index('3943726')

1323

In [22]:
[x for x in dft.index if x not in selected_columns]

['ALM12296078-20',
 'ALM18609-08',
 'ALM12247233-19',
 'ALM64446-14',
 'ALM29229-10',
 'ALM12714-08',
 'ALM12319442-20',
 'ALM40310-07']

In [140]:
def recalibrate_ddbb_vcf_intermediate(snp_matrix_ddbb_file, variant_folder, min_cov_low_freq=10):

    selected_columns = ['Position','N', 'Samples', 'CHROM','REF','POS', 'ALT']

    df = pd.read_csv(compare, sep="\t")
    df[['CHROM','REF','POS', 'ALT']] = df.Position.str.split("|",expand=True)

    df = df[ selected_columns + [ col for col in df.columns if col not in selected_columns]]

    dft = df.transpose()

    samples = [x for x in dft.index if x not in selected_columns]
    POSs = dft.loc['POS',:].tolist()
    REFs = dft.loc['REF',:].tolist()
    ALTs = dft.loc['ALT',:].tolist()

    n_samples = len(samples)

    header_df = dft.loc[selected_columns,:]
    samples_df = dft.drop(selected_columns, axis=0)

    # calculate the chunk size as an integer
    if (samples_df.shape[0] <= num_processes):
        chunks = [samples_df]
    else:
        chunk_size = int(samples_df.shape[0]/num_processes)
        chunks = [samples_df.iloc[samples_df.index[i:i + chunk_size]]
              for i in range(0, samples_df.shape[0], chunk_size)]

    # Iterate over non unanimous positions

    def review_with_vcf(df):
        return_df = df.reset_index(drop=True)
        for index, data_row in samples_df.iterrows():
            # Use enumerate to retrieve column index (column ondex + 3)
            # find positions with frequency >80% in mpileup execution
            # Returns ! for coverage 0
            new_presence_row = recheck_variant_rawvcf_intermediate(data_row, POSs, ALTs, variant_folder, min_cov_low_freq=10,)

            # new_presence_row_str = [str(x) for x in new_presence_row]

            # logger.info(whole_position + " " +
            #             (',').join(new_presence_row_str))

            # return_df.loc[index, 3:] = new_presence_row
            print(new_presence_row)

    
    for chunk in chunks:
        review_with_vcf(chunk)

    #with concurrent.futures.ThreadPoolExecutor(max_workers=num_processes) as executor:
    #    futures = []
    #    for chunk in chunks:
    #        future = executor.submit(review_with_vcf, chunk)
    #        futures.append(future)
    #    for future in concurrent.futures.as_completed(futures):
    #        logger.info(future.result())

    # pool = multiprocessing.Pool(processes=num_processes)
    # final_df = pd.concat(pool.map(review_with_vcf, chunks))
    # pool.close()
    # pool.join()

    def estract_sample_count(row):
        count_list = [i not in ['!', 0, '0'] for i in row[3:]]
        samples = np.array(df_matrix.columns[3:])
        # samples[np.array(count_list)] filter array with True False array
        return (sum(count_list), (',').join(samples[np.array(count_list)]))

    df_matrix[['N', 'Samples']] = df_matrix.apply(
        estract_sample_count, axis=1, result_type='expand')

    return df_matrix


In [143]:
def recheck_variant_rawvcf_intermediate(row, positions, alt_snps, variant_folder, min_cov_low_freq=10):
    """
    CU458896.1	3068036	.	G	A	262.784	.	AB=0.8;ABP=14.7363;AC=1;AF=0.5;AN=2;AO=12;CIGAR=1X;DP=15;DPB=15;DPRA=0;EPP=9.52472;EPPR=3.73412;GTI=0;LEN=1;MEANALT=1;MQM=60;MQMR=60;NS=1;NUMALT=1;ODDS=0.121453;PAIRED=0.333333;PAIREDR=0;PAO=0;PQA=0;PQR=0;PRO=0;QA=410;QR=108;RO=3;RPL=7;RPP=3.73412;RPPR=9.52472;RPR=5;RUN=1;SAF=4;SAP=5.9056;SAR=8;SRF=1;SRP=3.73412;SRR=2;TYPE=snp	GT:DP:AD:RO:QR:AO:QA:GL	0/1:15:3,12:3:108:12:410:-32.709,0,-5.55972
    """
    sample = row.name
    row_values = row.values
    print('POS', row_values[0:10])
    print('SAMPLE:', sample)
    return_list = [0] * len(positions)

    # Identify vcf
    variant_sample_folder = os.path.join(variant_folder, sample)
    filename = os.path.join(variant_sample_folder, "snps.raw.vcf")
                

    # Open file and retrieve output

    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('#CHROM'):
                headers = line.split("\t")
                vcf_sample = headers[-1].strip()
            elif not line.startswith('#'):
                line_split = line.split("\t")
                vcf_position = line_split[1]
                if vcf_position in positions:
                    position_index = positions.index(vcf_position)
                    #logger.info(
                    #    'recalibrating position: {} in sample: {}'.format(vcf_position, sample))
                    
                    # vcf_ref_base = line_split[3]
                    params = line_split[-2].split(":")
                    value_params = line_split[-1].split(":")
                    depth_index = params.index('DP')

                    vcf_reference = line_split[0]
                    vcf_alt_base = line_split[4]
                    vcf_depth = int(value_params[depth_index])
                    # ref_depth_indef = params.index('RO')
                    alt_depth_index = params.index('AO')
                    try:
                        vcf_alt_depth = int(value_params[alt_depth_index])
                    except:
                        vcf_alt_depth = int(
                            value_params[alt_depth_indef].split(',')[-1])
                        vcf_alt_base = vcf_alt_base.split(',')[-1]
                    vcf_alt_freq = vcf_alt_depth/vcf_depth
                        
                    logger.info('REC: POS: {},  ALT: {}, DP: {}, FREQ: {}\nORI==> POS: {}, ALT: {}'.format(
                        vcf_position, vcf_alt_base, vcf_depth, vcf_alt_freq, positions[position_index], alt_snps[position_index]))

    # if reference_id != vcf_reference:
    #     logger.info('ERROR: References are different')
    #     sys.exit(1)
    # elif (len(alt_snp) > 1 and str(position) == str(vcf_position)) or (len(vcf_reference) > 1 and str(position) == str(vcf_position)):
    #     if vcf_depth <= min_cov_low_freq and vcf_depth > 0:
    #         logger.debug('Position: {} LOWDEPTH: {}'.format(
    #             vcf_position, vcf_alt_freq))
    #         return '?'
    #     else:
    #         return vcf_alt_freq
    # elif str(position) == str(vcf_position) and alt_snp == vcf_alt_base:
    #     if vcf_depth <= min_cov_low_freq and vcf_depth > 0:
    #         logger.debug('Position: {} LOWDEPTH: {}'.format(
    #             vcf_position, vcf_alt_freq))
    #         return '?'
    #     if vcf_alt_freq > 0.1:
    #         logger.debug('Position: {} HTZ with freq > 0.1: {}'.format(
    #             vcf_position, vcf_alt_freq))
    #         return vcf_alt_freq
    # else:
    #     logger.debug('ELSE POS: {} SAMPLE: {}'.format(
    #         vcf_position, sample))
    #     return 0

In [144]:
recalibrate_ddbb_vcf_intermediate(compare, variant_folder, min_cov_low_freq=10)

RECALIBRATE: CHROM: MTB_anc POS: 2532,  ALT: T, DP: 87, FREQ: 1.0
ORI==> POS: 2532, ALT: T
RECALIBRATE: CHROM: MTB_anc POS: 9143,  ALT: T, DP: 248, FREQ: 0.9879032258064516
ORI==> POS: 9143, ALT: T
RECALIBRATE: CHROM: MTB_anc POS: 11370,  ALT: T, DP: 262, FREQ: 0.9885496183206107
ORI==> POS: 11370, ALT: T
RECALIBRATE: CHROM: MTB_anc POS: 12020,  ALT: A, DP: 170, FREQ: 1.0
ORI==> POS: 12020, ALT: A
RECALIBRATE: CHROM: MTB_anc POS: 13460,  ALT: A, DP: 281, FREQ: 1.0
ORI==> POS: 13460, ALT: A
RECALIBRATE: CHROM: MTB_anc POS: 14051,  ALT: C, DP: 223, FREQ: 1.0
ORI==> POS: 14051, ALT: C
RECALIBRATE: CHROM: MTB_anc POS: 14401,  ALT: A, DP: 238, FREQ: 0.9957983193277311
ORI==> POS: 14401, ALT: A
RECALIBRATE: CHROM: MTB_anc POS: 15117,  ALT: C, DP: 279, FREQ: 1.0
ORI==> POS: 15117, ALT: C
RECALIBRATE: CHROM: MTB_anc POS: 21819,  ALT: A, DP: 24, FREQ: 1.0
ORI==> POS: 21819, ALT: A
RECALIBRATE: CHROM: MTB_anc POS: 23174,  ALT: C, DP: 200, FREQ: 0.995
ORI==> POS: 23174, ALT: C
RECALIBRATE: CHROM:

NameError: name 'alt_depth_indef' is not defined